In [1]:
import os
import librosa
import numpy as np
import json

In [2]:
def extract_features(file_name, sr=44100, duration=30):
    # โหลดไฟล์เสียงและแซมปลิงเรทที่กำหนด
    y, _ = librosa.load(file_name, sr=sr, duration=duration)
    
    # สร้าง dictionary สำหรับเก็บ features ของไฟล์เสียง
    features = {}
    
    # คำนวณ MFCC (Mel-Frequency Cepstral Coefficients) และเก็บค่าเฉลี่ยใน axis 0
    features['mfcc'] = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0).tolist()
    
    # คำนวณ Spectral Centroid และเก็บค่าเฉลี่ยใน axis 0
    features['spectral_centroid'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T, axis=0).tolist()
    
    # คำนวณ Zero-Crossing Rate และเก็บค่าเฉลี่ยใน axis 0
    features['zero_crossing_rate'] = np.mean(librosa.feature.zero_crossing_rate(y).T, axis=0).tolist()
    
    # คำนวณ Spectral Bandwidth และเก็บค่าเฉลี่ยใน axis 0
    features['spectral_bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr).T, axis=0).tolist()
    
    # คำนวณ Chroma Feature และเก็บค่าเฉลี่ยใน axis 0
    features['chroma_stft'] = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0).tolist()
    
    # คำนวณ Mel-Spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    
    # แปลง Mel-Spectrogram เป็น dB และเก็บค่าเฉลี่ยใน axis 0
    features['mel_spectrogram'] = np.mean(librosa.power_to_db(mel_spectrogram).T, axis=0).tolist()
    
    # ส่งคืน dictionary ที่มี features ของไฟล์เสียง
    return features


In [3]:
# ฟังก์ชันสำหรับเตรียมข้อมูลที่จำเป็นสำหรับการฝึกอบรมโมเดล
def prepare_data(data_path='../you_path', sr=44100, duration=30):
    # รายการของแนวเพลงที่จะประมวลผล
    genres = ['Classical', 'Country', 'HipHop', 'Jazz', 'Metal', 'Rock']
    
    # พจนานุกรมเพื่อเก็บคุณลักษณะที่ดึงออกมาสำหรับแต่ละแนวเพลง
    data = {}

    # วน loop ผ่านแต่ละแนวเพลง
    for genre in genres:
        # พาธไปยังไดเรกทอรีที่มีไฟล์เสียงของแต่ละแนวเพลง
        genre_path = os.path.join(data_path, genre)
        # กำหนดค่าเริ่มต้นในการเก็บลิสต์สำหรับคุณลักษณะของแต่ละแนวเพลง
        data[genre] = []
        
        # วน loop ผ่านทุกไฟล์ในไดเรกทอรีของแนวเพลง
        for file_name in os.listdir(genre_path):
            # กรองเฉพาะไฟล์ที่มีนามสกุล .wav เท่านั้น
            if file_name.endswith('.wav'):
                # พาธเต็มไปยังไฟล์เสียง
                file_path = os.path.join(genre_path, file_name)
                # ดึงคุณลักษณะหรือ feature ออกมาจากไฟล์เสียง
                features = extract_features(file_path, sr, duration)
                # เพิ่มคุณลักษณะที่ดึงออกมาไปยังลิสต์ของแนวเพลงนั้น
                data[genre].append(features)

    # บันทึกคุณลักษณะที่ดึงออกมาลงในไฟล์ JSON
    with open('../you_path.json', 'w') as fp:
        json.dump(data, fp, indent=4)


In [4]:
# ฟังก์ชันสำหรับตรวจสอบคุณภาพข้อมูลคุณลักษณะที่ถูกดึงออกมา
def check_features(data):
    issues = []
    expected_lengths = {
        'mfcc': 13,
        'spectral_centroid': 1,
        'zero_crossing_rate': 1,
        'spectral_bandwidth': 1,
        'chroma_stft': 12,
        'mel_spectrogram': 128
    }
    
    # วนลูปผ่านแต่ละแนวเพลงและลิสต์คุณลักษณะ
    for genre, features_list in data.items():
        for i, features in enumerate(features_list):
            print(f"ตรวจสอบ {genre} ตัวอย่างที่ {i + 1}")
            # ตรวจสอบแต่ละคุณลักษณะภายในพจนานุกรมคุณลักษณะ
            for feature_name, feature_values in features.items():
                if not isinstance(feature_values, list):
                    issues.append(f"{genre} ตัวอย่างที่ {i + 1}: {feature_name} ไม่ใช่ลิสต์")
                elif len(feature_values) != expected_lengths[feature_name]:
                    issues.append(f"{genre} ตัวอย่างที่ {i + 1}: {feature_name} มี {len(feature_values)} องค์ประกอบ คาดหวัง {expected_lengths[feature_name]} องค์ประกอบ")
                else:
                    print(f"{feature_name}: {len(feature_values)} องค์ประกอบ")
    
    # รายงานผล
    if not issues:
        print("\nคุณลักษณะทั้งหมดมีรูปแบบที่ถูกต้องและมีจำนวนองค์ประกอบตามที่คาดหวัง")
    else:
        print("\nพบปัญหาดังนี้:")
        for issue in issues:
            print(issue)

In [5]:
# เริ่มการเตรียมข้อมูล
prepare_data()

In [6]:
# โหลดข้อมูลจากไฟล์ JSON
with open('../you_path', 'r') as fp: #Features_Full_Song
    data = json.load(fp)

In [7]:
# ตรวจสอบข้อมูลใน JSON
check_features(data)

Checking Classical sample 1
mfcc: 13 elements
spectral_centroid: 1 elements
zero_crossing_rate: 1 elements
spectral_bandwidth: 1 elements
chroma_stft: 12 elements
mel_spectrogram: 128 elements
Checking Classical sample 2
mfcc: 13 elements
spectral_centroid: 1 elements
zero_crossing_rate: 1 elements
spectral_bandwidth: 1 elements
chroma_stft: 12 elements
mel_spectrogram: 128 elements
Checking Classical sample 3
mfcc: 13 elements
spectral_centroid: 1 elements
zero_crossing_rate: 1 elements
spectral_bandwidth: 1 elements
chroma_stft: 12 elements
mel_spectrogram: 128 elements
Checking Classical sample 4
mfcc: 13 elements
spectral_centroid: 1 elements
zero_crossing_rate: 1 elements
spectral_bandwidth: 1 elements
chroma_stft: 12 elements
mel_spectrogram: 128 elements
Checking Classical sample 5
mfcc: 13 elements
spectral_centroid: 1 elements
zero_crossing_rate: 1 elements
spectral_bandwidth: 1 elements
chroma_stft: 12 elements
mel_spectrogram: 128 elements
Checking Classical sample 6
mfcc: 1